In [1]:
import sys
sys.path.append("../Simulations/")

from units import *
import matplotlib.pyplot as plt
from tqdm import *
from scipy.integrate import nquad, quad
from scipy.special import erf, jn, jv, kn
import mpmath as mp

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [25]:
def c200_SC(M200):
    """ Concentration-mass relation according to Sanchez-Conde&Prada14
    """
    x=np.log(M200*h) # Given in terms of M_s/h in S-C&P paper
    pars=[37.5153, -1.5093, 1.636e-2, 3.66e-4, -2.89237e-5, 5.32e-7][::-1]
    return np.polyval(pars, x)

def get_rs_rhos_NFW(M200):
    """ Get NFW scale radius and density
    """
    c200 = c200_SC(M200/M_s)
    r200 = (M200/(4/3.*np.pi*200*rho_c))**(1/3.)
    rho_s = M200/(4*np.pi*(r200/c200)**3*(np.log(1 + c200) - c200/(1 + c200)))
    r_s = r200/c200
    return r_s, rho_s

def F(x):
    """ Helper function for NFW deflection, from astro-ph/0102341
    """
    if x > 1:
        return mp.atan(mp.sqrt(x**2-1))/(mp.sqrt(x**2 - 1))
    elif x == 1:
        return 1
    elif x < 1:
        return mp.atanh(mp.sqrt(1-x**2))/(mp.sqrt(1-x**2))
    

In [19]:
def MNFWdivM0(x):
    """ Enclosed mass in cylinder, NFW profile
    """
    return (mp.log(x/2) + F(x))

In [35]:
def MNFWdivM0_integ(theta_s, l):
    return mp.quadosc(lambda theta: MNFWdivM0(theta/theta_s)*mp.j1(l*theta), [0, mp.inf], period=2*mp.pi/l)

In [83]:
l_min, l_max = 1, 1000
n_l = 50
l_ary = np.logspace(np.log10(l_min), np.log10(l_max), n_l)

theta_s_min, theta_s_max = 0.0001, 0.5
n_theta_s = 20
theta_s_ary = np.linspace(theta_s_min, theta_s_max, n_theta_s)

MjdivM0_integ_ary = np.zeros((n_theta_s, n_l))

for itheta_s, theta_s in enumerate(tqdm_notebook(theta_s_ary)):
    for il, l in enumerate(tqdm_notebook(l_ary)):
        MjdivM0_integ_ary[itheta_s, il] = MNFWdivM0_integ(theta_s, l)

In [46]:
from scipy.interpolate import interp2d

In [76]:
MjdivM0_integ_interp = interp2d(np.log10(l_ary), theta_s_ary, MjdivM0_integ_ary, kind='quintic')

In [81]:
MNFWdivM0_integ(0.001, 10)

mpf('0.40433858274066992')

In [82]:
MjdivM0_integ_interp(0.001, np.log10(10))[0]

0.6698365444323445

In [72]:
def Cl_NFW_new(M200, Dl, v, l):
    r_s, rho_s = get_rs_rhos_NFW(M200)
    M0 = 4*np.pi*r_s**3*rho_s
    pref = GN**2*v**2*8*np.pi*l**2/Dl**4
    theta_s = r_s/Dl
#     MjdivM0 = mp.quadosc(lambda theta: MNFWdivM0(theta/theta_s)*mp.j1(l*theta), [0, mp.inf], period=2*mp.pi/l)
    MjdivM0 = MjdivM0_integ_interp(theta_s, np.log10(l))[0]

    return pref*M0**2*MjdivM0**2


In [73]:
M0 = 10**8*M_s
R0 = R0_VL(M0)
Dl = 10*kpc
v = 1e-3

l = 1

In [74]:
%%timeit
Cl_NFW_new(M0, Dl, v, l)/(1e-6*asctorad/Year)**2

70.8 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
